# Проект по итогам курса: Анализ данных SRR12905311
1. Описание исследования


Исследование:

Данный набор данных SRR12905311 взят из базы SRA и содержит секвенированные риды бактерии Escherichia coli (или другого микроорганизма, в зависимости от конкретных данных). Обычно такие данные используются для изучения генетической вариации в штаммах бактерий, например, для поиска мутаций, связанных с устойчивостью к антибиотикам или адаптацией к окружающей среде.

Источник:  

SRA: SRR12905311

GEO: GSM4862160 (ссылка на соответствующий GEO-объект, подтверждающий тип исследования и условия)

Общий смысл:

Исследование направлено на выявление генетических мутаций в бактериальных геномах, что помогает понять механизмы адаптации и устойчивости.

2. Загрузка и подготовка данных

Шаг 1. Скачать данные из SRA

In [23]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MedicalNER") \
    .master("local[*]") \
    .config("spark.driver.memory", "4G") \
    .getOrCreate()

import sparknlp
sparknlp.start()

# Ваш текст
medical_text = """
Fibrodysplasia ossificans progressiva (FOP) is a rare and progressively debilitating disorder
affecting 1 in 2 million individuals. It is characterized by the progressive ossification of soft tissues into ectopic bone and congenital malformations of the great toes.
FOP leads to significant disability and can result in death due to thoracic insufficiency syndrome. These case reports examine the presentation, diagnosis, and management
of FOP, highlighting the diagnostic challenges inherent in managing such rare conditions because of their unique clinical features. They underscore the need for increased
awareness among healthcare practitioners to ensure early diagnosis and the implementation of effective management strategies.
"""

# Загрузка модели
try:
    ner_model = NerDLModel.pretrained("ner_jsl", "en")
    print("Модель 'ner_jsl' успешно загружена.")
except Exception as e:
    print(f"Ошибка при загрузке модели 'ner_jsl': {e}")

if ner_model:
    # Создаем этапы обработки
    document_assembler = DocumentAssembler() \
        .setInputCol("text") \
        .setOutputCol("document")

    sentence_detector = SentenceDetector() \
        .setInputCols(["document"]) \
        .setOutputCol("sentences")

    tokenizer = Tokenizer() \
        .setInputCols(["sentences"]) \
        .setOutputCol("tokens")

    ner_tagger = ner_model.setInputCols(["sentences", "tokens"]) \
                          .setOutputCol("ner")

    pipeline = Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        ner_tagger
    ])

    data = spark.createDataFrame([[medical_text]]).toDF("text")

    model_pipeline = pipeline.fit(data)
    result_df = model_pipeline.transform(data)

    result_df.selectExpr("explode(ner) as ner_result") \
        .select("ner_result.*") \
        .show(truncate=False)
else:
    print("Модель не была успешно загружена.")

ner_jsl download started this may take some time.
Can not find the model to download please check the name!
Модель 'ner_jsl' успешно загружена.
Модель не была успешно загружена.


In [30]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql.functions import explode, col

# Создаем компоненты
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

tokenizer = Tokenizer() \
    .setInputCols(["documents"]) \
    .setOutputCol("tokens")

# Загружаем модель NER
ner_model = BertForTokenClassification.pretrained("bert_finetuned_ner_clinical_plncmm_large_22", "en") \
    .setInputCols(["documents", "tokens"]) \
    .setOutputCol("ner")

# Создаем pipeline
pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    ner_model
])

# Текст статьи или фрагмента
text = """
A 35-year-old woman diagnosed with fibrodysplasia ossificans progressiva (FOP) presented with progressive heterotopic ossification. The condition is characterized by abnormal bone formation outside the skeleton, often following trauma. Treatment options remain limited.
"""

# Создаем DataFrame
data = spark.createDataFrame([[text]]).toDF("text")

# Обучение и применение модели
pipeline_model = pipeline.fit(data)
result = pipeline_model.transform(data)

# Обработка результатов
# Распаковать массив сущностей
entities = result.select(explode(col("ner")).alias("entity")) \
    .select(
        col("entity.result").alias("entity_text"),
        col("entity.metadata").getItem("entity").alias("entity_type")
    )

entities.show(truncate=False)

bert_finetuned_ner_clinical_plncmm_large_22 download started this may take some time.
Approximate size to download 390.7 MB
[OK!]
+-----------+-----------+
|entity_text|entity_type|
+-----------+-----------+
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|B-Disease  |NULL       |
|I-Disease  |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
|O          |NULL       |
+-----------+-----------+
only showing top 20 rows

